In [1]:
using Pkg; Pkg.activate("/home/dhairyagandhi96/temp/model-zoo/script/.."); Pkg.status();

    Status `~/temp/model-zoo/Project.toml`
  [1520ce14]   AbstractTrees v0.2.1
  [fbb218c0] ↑ BSON v0.2.3 ⇒ v0.2.4
  [54eefc05]   Cascadia v0.4.0
  [8f4d0f93]   Conda v1.3.0
  [864edb3b] ↑ DataStructures v0.17.0 ⇒ v0.17.5
  [31c24e10] ↑ Distributions v0.21.3 ⇒ v0.21.5
  [587475ba]   Flux v0.9.0
  [708ec375]   Gumbo v0.5.1
  [b0807396]   Gym v1.1.3
  [cd3eb016] ↑ HTTP v0.8.6 ⇒ v0.8.7
  [6218d12a]   ImageMagick v0.7.5
  [916415d5]   Images v0.18.0
  [e5e0dc1b]   Juno v0.7.2
  [ca7b5df7]   MFCC v0.3.1
  [dbeba491] + Metalhead v0.4.0 #c4d1eba (https://github.com/FluxML/Metalhead.jl.git)
  [91a5bcdd] ↑ Plots v0.26.3 ⇒ v0.27.0
  [2913bbd2]   StatsBase v0.32.0
  [98b73d46]   Trebuchet v0.1.0
  [8149f6b0] ↑ WAV v1.0.2 ⇒ v1.0.3
  [10745b16]   Statistics 
  [4ec0a83e]   Unicode 


Based on https://arxiv.org/abs/1409.0473

In [2]:
using Flux: flip, crossentropy, reset!, throttle

include("0-data.jl")

Nin = length(alphabet)
Nh = 30 # size of hidden layer

30

A recurrent model which takes a token and returns a context-dependent
annotation.

In [3]:
forward  = LSTM(Nin, Nh÷2)
backward = LSTM(Nin, Nh÷2)
encode(tokens) = vcat.(forward.(tokens), flip(backward, tokens))

alignnet = Dense(2Nh, 1)
align(s, t) = alignnet(vcat(t, s .* trues(1, size(t, 2))))

align (generic function with 1 method)

A recurrent model which takes a sequence of annotations, attends, and returns
a predicted output token.

In [4]:
recur   = LSTM(Nh+length(phones), Nh)
toalpha = Dense(Nh, length(phones))

function asoftmax(xs)
  xs = [exp.(x) for x in xs]
  s = sum(xs)
  return [x ./ s for x in xs]
end

function decode1(tokens, phone)
  weights = asoftmax([align(recur.state[2], t) for t in tokens])
  context = sum(map((a, b) -> a .* b, weights, tokens))
  y = recur(vcat(Float32.(phone), context))
  return softmax(toalpha(y))
end

decode(tokens, phones) = [decode1(tokens, phone) for phone in phones]

decode (generic function with 1 method)

The full model

In [5]:
state = (forward, backward, alignnet, recur, toalpha)

function model(x, y)
  ŷ = decode(encode(x), y)
  reset!(state)
  return ŷ
end

loss(x, yo, y) = sum(crossentropy.(model(x, yo), y))

evalcb = () -> @show loss(data[500]...)
opt = ADAM()

Flux.train!(loss, params(state), data, opt, cb = throttle(evalcb, 10))

loss(data[500]...) = 30.819227f0 (tracked)
loss(data[500]...) = 18.678741f0 (tracked)
loss(data[500]...) = 16.73465f0 (tracked)
loss(data[500]...) = 15.124601f0 (tracked)
loss(data[500]...) = 14.582631f0 (tracked)
loss(data[500]...) = 13.131728f0 (tracked)
loss(data[500]...) = 13.291695f0 (tracked)
loss(data[500]...) = 12.551632f0 (tracked)
loss(data[500]...) = 13.084431f0 (tracked)
loss(data[500]...) = 13.316135f0 (tracked)
loss(data[500]...) = 14.365998f0 (tracked)
loss(data[500]...) = 14.262397f0 (tracked)
loss(data[500]...) = 15.117974f0 (tracked)
loss(data[500]...) = 17.314672f0 (tracked)


Prediction

In [6]:
using StatsBase: wsample

function predict(s)
  ts = encode(tokenise(s, alphabet))
  ps = Any[:start]
  for i = 1:50
    dist = decode1(ts, onehot(ps[end], phones))
    next = wsample(phones, vec(Tracker.data(dist)))
    next == :end && break
    push!(ps, next)
  end
  return ps[2:end]
end

predict("PHYLOGENY")

14-element Array{Any,1}:
 :OW1
 :K  
 :AH0
 :L  
 :OW0
 :IH0
 :V  
 :AH0
 :L  
 :IH0
 :N  
 :IY0
 :IY2
 :N  